In [2]:

import numpy as np
import pandas as pd
import os
import re
from dateutil.parser import parse


In [3]:

# === Load and clean ===
source_path = "../ncerdc-data/"
prefix = "curtest"  

sas_files = [f for f in os.listdir(source_path) if f.startswith(prefix) and f.endswith(".sas7bdat")]
df_list = [pd.read_sas(os.path.join(source_path, f), format="sas7bdat", encoding="latin1") for f in sas_files]
big_df = pd.concat(df_list, ignore_index=True)

# Standardize column names
caps_cols = [col for col in big_df.columns if col.isupper()]
for col in caps_cols:
    lower_col = col.lower()
    if lower_col in big_df.columns:
        big_df[lower_col] = big_df[lower_col].combine_first(big_df[col])
        big_df = big_df.drop(columns=[col])
    else:
        big_df = big_df.rename(columns={col: lower_col})

big_df = big_df[['year','mastid','test_id','score']]

# Clean scores
big_df["score"] = big_df["score"].replace("NULL", np.nan)
big_df.dropna(subset=["score"], inplace=True)
big_df["score"] = pd.to_numeric(big_df["score"], errors="coerce").astype("float")

# === Handle ACT differently ===
# Flag ACT tests
is_act = big_df["test_id"].str.startswith("AC")

# Percentiles for non-ACT
big_df.loc[~is_act, "score_value"] = (
    big_df.loc[~is_act]
    .groupby(["year", "test_id"])["score"]
    .rank(pct=True) * 100
)

# Raw scores for ACT
big_df.loc[is_act, "score_value"] = big_df.loc[is_act, "score"]

# Drop unnecessary cols
big_df = big_df.drop(columns=["score", "year"])
big_df.dropna(subset=["score_value"], inplace=True)

# === Pivot ===
pivot_df = big_df.pivot_table(
    index="mastid",
    columns="test_id",
    values="score_value",
    aggfunc="first"
).reset_index()

# Drop unwanted test_id groups
pivot_df = pivot_df.loc[:, ~pivot_df.columns.str.startswith(("X", "W", "U", "G", "C", "P"))]
pivot_df = pivot_df.drop(columns=['RD3A','RD3B','RD3R','ALG1','ALG2','ENG2','ENGL','MTH3'], errors="ignore")

# === Split ACT vs others ===
a_cols = [c for c in pivot_df.columns if c.startswith("AC")] + ["mastid"]
act = pivot_df[a_cols]

other_cols = [c for c in pivot_df.columns if c not in a_cols]
df_other = pivot_df[["mastid"] + other_cols]

# === Save ===
df_other.to_csv("data/interim/curtest_master.csv", index=False)
act.to_csv("data/interim/act.csv", index=False)


masterbuild is such a pain to do again, exercise for the reader
lea, gender, ethnicity, eds, aig, grade and year are contained in masterbuild

In [4]:
#sat
sat = pd.DataFrame()
for year in range(18, 23):  # 23 because range is exclusive at the end
    filename = f"../ncerdc-data/collegeboard{year}pub.sas7bdat"
    try:
        # Read CSV file
        df = pd.read_sas(filename)
        
        # Keep only desired columns
        cols = [c for c in df.columns if c.startswith("SAT_Total_Score_HC") or c == "mastid" or c.startswith("SAT_Math_Score_HC") or c.startswith("SAT_EBRW_Score_HC")]

        df = df[cols]        
        df.dropna(inplace=True)
        
        # Append to the main DataFrame
        sat = pd.concat([sat, df], ignore_index=True)
        
    except FileNotFoundError:
        print(f"File {filename} not found. Skipping.")
    

# Optional: Display or save the combined DataFrame
# sat.to_csv("combined_sat_data.csv", index=False)
# Keep the highest SAT_Total_Score_HC for each mastid
sat = sat.sort_values('SAT_Total_Score_HC', ascending=False).drop_duplicates(subset='mastid', keep='first')

sat.to_csv("data/interim/sat.csv",index=False)

In [5]:
folder_path = "../ncerdc-data"  # Change this if your files are in a different directory


# Loop through files in the directory
for filename in os.listdir(folder_path):
    match = re.match(r"gpa(\d{4})\.sas7bdat", filename)
    if match:
        year = match.group(1)
        var_name = f"gpa{year}"
        file_path = os.path.join(folder_path, filename)
        df = pd.read_sas(file_path)
        globals()[var_name] = df
        
gpa_vars = [var for var in globals() if var.startswith("gpa") and isinstance(globals()[var], pd.DataFrame) and var != "gpa_master"]


gpa_master = pd.concat([globals()[var] for var in gpa_vars], ignore_index=True)
gpa_master = gpa_master.map(lambda x: x[2:-1] if isinstance(x, str) and x.startswith('b') else x)


# Combine the columns, prioritizing non-null values in 'bound_for'
gpa_master['bound_for_combined'] = gpa_master['bound_for'].combine_first(gpa_master['BOUND_FOR'])

# Drop the original columns
gpa_master.drop(columns=['bound_for', 'BOUND_FOR'], inplace=True)

# Rename to a standard name (optional)
gpa_master.rename(columns={'bound_for_combined': 'bound_for'}, inplace=True)


gpa_master.drop(columns=['DIPLOMA_MET','UNWEIGHTED_RANK_DATE','WEIGHTED_RANK_DATE','diploma_type','DIPLOMA_TYPE'], inplace=True)


# def fast_parse_dates(series):
#     # Try general parse first (fastest)
#     parsed = pd.to_datetime(series, errors='coerce')

#     # Optionally: fallback to common format if still missing
#     fallback = pd.to_datetime(series, format='%m/%d/%Y', errors='coerce')
#     parsed = parsed.fillna(fallback)

#     return parsed.dt.year


# gpa_master['entry_year'] = fast_parse_dates(gpa_master['NINTHGRADEENTRY'])
# gpa_master['grad_year'] = fast_parse_dates(gpa_master['DIPLOMA_ISSUED'])

def clean_bytes(df):
    """
    Convert byte-string columns (like b'ABC') into normal Python strings.
    Applies only to object or string dtypes.
    """
    for col in df.select_dtypes(include=["object", "string"]):
        df[col] = df[col].apply(
            lambda x: x.decode("utf-8").strip() if isinstance(x, bytes) else x
        )
    return df

gpa_master = gpa_master[['mastid','bound_for','gpa_unweighted','gpa_weighted']]
gpa_master = clean_bytes(gpa_master)
gpa_master = gpa_master[(gpa_master['gpa_unweighted'] <= 4) & (gpa_master['gpa_weighted'] <= 6)]
gpa_master.to_csv("data/interim/gpa_master.csv",index=False)